In [1]:
from rdflib import Namespace, Graph, Literal, URIRef
import re
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes
from buildingmotif.model_builder import TemplateBuilderContext
import os 
import sys 
# %%
# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://")

# create the model w/ a namespace
WBS = Namespace("urn:ex/")
S223 = Namespace("http://data.ashrae.org/standard223#")
wbs = Model.create(WBS)
bind_prefixes(wbs.graph)
wbs.graph.bind("wbs", WBS)
wbs.graph.bind("s223", S223)
wbs.graph.bind("nawi", Namespace("urn:nawi-water-ontology#"))
things = []

In [2]:
sys.path.append('..')
from utils import * # ModelBuilder and vars

In [3]:
# %%
templates = Library.load(directory='../templates')
s223 = Library.load(ontology_graph="../../s223/collections/MODEL_SP223_all-v1.0.ttl")
templates.get_templates()

/Users/fletch/Applications/miniconda3/lib/python3.10/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))
2025-06-03 15:41:19,192 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/vocab/enumeration (No row was found when one was required). Check this is loaded into BuildingMOTIF
2025-06-03 15:41:19,193 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/constant (No row was found when one was required). Check this is loaded into BuildingMOTIF
2025-06-03 15:41:19,193 | root |  WARNING: An ontology could not resolve a dependency on http://www.w3.org/ns/shacl# (No row was found when one was required). Check this is loaded into BuildingMOTIF
2025-06-03 15:41:19,194 | root |  WARNING: An ontology could not resolve a depe

[Template(_id=1, _name='brine', body=<Graph identifier=556cc678-ee1a-4a62-9ed5-1ef725ff909f (<class 'rdflib.graph.Graph'>)>, optional_args=['constituent-salt', 'constituent-water'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x1039d1ea0>),
 Template(_id=2, _name='constituent-salt', body=<Graph identifier=56a659c4-ac3d-4f72-b349-96530081c975 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x1039d1ea0>),
 Template(_id=3, _name='constituent-water', body=<Graph identifier=a3dbf37f-a34d-43e1-b225-43b206b9f576 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x1039d1ea0>),
 Template(_id=4, _name='ph', body=<Graph identifier=4facf1aa-a440-469b-94b4-ea9f110eef13 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x1039d1ea0>),
 Template(_id=5, 

In [4]:
builder = ModelBuilder(ns = WBS, bm_instance= wbs, templates = templates)

In [5]:
# TODO: how to add column 53, "GAC-UVT (%)" between GAC and UV system?
h2o2_tank = builder.create_equipment('h2o2-tank', 'tank', role=S223['Role-Supply'])
h2o2_dosing_pump = builder.create_equipment('h2o2-pump', 'pump', role=S223['Role-Supply'])
builder.connect_equipment(h2o2_tank, h2o2_dosing_pump)
h2o2_dosing_pump_props = builder.add_properties(h2o2_dosing_pump['name'], ['status', 'flow-rate', 'speed-command'], ['145', '94', '68'])
lpuv_bypass_valve = builder.create_equipment('lpuv-bypass-valve', 'valve')
builder.connect_equipment(h2o2_dosing_pump, lpuv_bypass_valve)
lpuv_bypass_outlet = builder.add_connection_point(lpuv_bypass_valve, 'outlet-cp')
# TODO: should low pressure UV be a subclass of standard UV reactor? Right now all 4 are the same class
lpuv_reactor_1 = builder.create_equipment('lpuv-1', 'uv-reactor')
# TODO: column 26 is "GAC-C3 Flow (GPM)", which doesn't seem to match the LPUV reactor...
# lpuv_reactor_1_props = builder.add_properties(lpuv_reactor_1['name'], ['status'], ['26'])
lpuv_reactor_2 = builder.create_equipment('lpuv-2', 'uv-reactor')
lpuv_reactor_2_props = builder.add_properties(lpuv_reactor_2['name'], ['status'], ['141']) #, '107'])
lpuv_reactor_3 = builder.create_equipment('lpuv-3', 'uv-reactor')
# TODO: column 107 is "SCADA Enclosure Door", which doesn't seem to match the LPUV reactor...
# lpuv_reactor_3_props = builder.add_properties(lpuv_reactor_1['name'], ['status'], ['107'])
lpuv_1_to_2 = builder.connect_equipment(lpuv_reactor_1, lpuv_reactor_2)
lpuv_2_to_3 = builder.connect_equipment(lpuv_reactor_2, lpuv_reactor_3)
# TODO: On / Off and LED % do not have a column numbers
uv_led_reactor = builder.create_equipment('uv-led', 'uv-reactor')
builder.connect_equipment(lpuv_bypass_valve, uv_led_reactor, source_cp=lpuv_bypass_outlet)
uv_outlet_valve = builder.create_equipment('uv-out-valve', 'valve')
builder.connect_equipment(lpuv_reactor_3, uv_outlet_valve)
uv_led_outlet = builder.add_connection_point(uv_outlet_valve, 'inlet-cp')
builder.connect_equipment(uv_led_reactor, uv_outlet_valve, target_cp=uv_led_outlet)

EQUIP HAS ALL NEEDED PROPERTIES:  h2o2-tank
EQUIP HAS ALL NEEDED PROPERTIES:  h2o2-pump
EQUIP HAS ALL NEEDED PROPERTIES:  lpuv-bypass-valve
CP HAS ALL NEEDED PROPERTIES:  urn:ex/lpuv-bypass-valve-outlet-cp
EQUIP HAS ALL NEEDED PROPERTIES:  lpuv-1
EQUIP HAS ALL NEEDED PROPERTIES:  lpuv-2
EQUIP HAS ALL NEEDED PROPERTIES:  lpuv-3
EQUIP HAS ALL NEEDED PROPERTIES:  uv-led
EQUIP HAS ALL NEEDED PROPERTIES:  uv-out-valve
CP HAS ALL NEEDED PROPERTIES:  urn:ex/uv-out-valve-inlet-cp


{'in': rdflib.term.URIRef('urn:ex/uv-led-out'),
 'out': rdflib.term.URIRef('urn:ex/uv-out-valve-inlet-cp'),
 'name': rdflib.term.URIRef('urn:ex/conn-uv-led-to-uv-out-valve')}

In [6]:
# TODO: how do we want to represent dosing points?
chlorine_dosing_point = builder.create_equipment(
    'chlorine-dose-point', 
    'junction', 
    optional_dict={'in1': 'dose-inlet', 'in2': 'uv-inlet', 'out1': 'chlorine-dose-out', 'medium': S223['Medium-Chlorine']}
)
builder.connect_equipment(uv_outlet_valve, chlorine_dosing_point)
chlorine_tank = builder.create_equipment('chlorine-tank', 'tank', role=S223['Role-Supply'])
chlorine_pump = builder.create_equipment('chlorine-pump', 'tank', role=S223['Role-Supply'])
builder.connect_equipment(chlorine_tank, chlorine_pump)
chlorine_pump_props = builder.add_properties(chlorine_pump['out'], ['flow-rate', 'speed-command'], ['93', '67'])
builder.connect_equipment(chlorine_pump, chlorine_dosing_point)
chlorine_contactor = builder.create_equipment('chlorine-contactor', 'contact-chamber')
builder.connect_equipment(chlorine_dosing_point, chlorine_contactor)
effluent_valve = builder.create_equipment('potable-effluent-valve', 'valve')
effluent_pipe = builder.connect_equipment(chlorine_contactor, effluent_valve)
# TODO: effluent pressure indicator (PI / eff) does not have a column number
# Note: omitted column 46 since that is the Multi-TOC. I.e., the TOC measurement from different sampling points combined
effluent_pipe_props = builder.add_properties(effluent_pipe['name'], ['toc-ppm', 'ph', 'turbidity', 'cl2-mgL'], [75, 39, 35, 37])

EQUIP HAS ALL NEEDED PROPERTIES:  chlorine-dose-point
EQUIP HAS ALL NEEDED PROPERTIES:  chlorine-tank
EQUIP HAS ALL NEEDED PROPERTIES:  chlorine-pump
EQUIP HAS ALL NEEDED PROPERTIES:  chlorine-contactor
EQUIP HAS ALL NEEDED PROPERTIES:  potable-effluent-valve


In [7]:
builder.graph.serialize("disinfect-model.ttl", format="turtle")

<Graph identifier=b0974079-e049-4354-8a57-82c71fd749d5 (<class 'rdflib.graph.Graph'>)>